# Demo : VersionedData Zarr storage

In [6]:
import sys
sys.path.append('../')
import numpy as np
import zarr

### Import our library
VersionedData is a Zarr Storage class

In [7]:
from versionedzarrlib import VersionedData

In [8]:
path = "/Users/Marwan/Desktop/activelearning/data/versioned_data"
dims = (600, 600, 600)
chunk_size = (128, 128, 128)

#### Create new data

In [9]:
data = VersionedData(path=path, shape=dims, raw_chunk_size=chunk_size)
data.create(overwrite=True)

Grid dimensions: [5, 5, 5]
Start file creation ..
File already exists ! 
File will be deleted !
{'zarr_format': 2, 'shape': (600, 600, 600), 'chunks': (128, 128, 128), 'dtype': dtype('int8'), 'compressor': {'blocksize': 0, 'clevel': 5, 'cname': 'lz4', 'id': 'blosc', 'shuffle': 1}, 'fill_value': 0, 'order': 'C', 'filters': None, 'total_chunks': 1}
Dataset created!


#### Open in Zarrr

In [10]:
z = zarr.open(store=data)
z.info

Type,zarr.core.Array
Data type,int8
Shape,"(600, 600, 600)"
Chunk shape,"(128, 128, 128)"
Order,C
Read-only,False
Compressor,"Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0)"
Store type,versionedzarrlib.VersionedZarrData.VersionedData
No. bytes,216000000 (206.0M)
No. bytes stored,388
Storage ratio,556701.0


In [ ]:
# data.checkou

In [6]:
z[500, 500, 500] = 5
z[10,10,10] = 10

{'chunks': [128, 128, 128], 'compressor': {'blocksize': 0, 'clevel': 5, 'cname': 'lz4', 'id': 'blosc', 'shuffle': 1}, 'dtype': '|i1', 'fill_value': 0, 'filters': None, 'order': 'C', 'shape': [600, 600, 600], 'total_chunks': 1, 'zarr_format': 2}
total: 1
New file /Users/Marwan/Desktop/activelearning/data/versioned_data/raw/1
Writing (3, 3, 3)
{'chunks': [128, 128, 128], 'compressor': {'blocksize': 0, 'clevel': 5, 'cname': 'lz4', 'id': 'blosc', 'shuffle': 1}, 'dtype': '|i1', 'fill_value': 0, 'filters': None, 'order': 'C', 'shape': [600, 600, 600], 'total_chunks': 2, 'zarr_format': 2}
total: 2
New file /Users/Marwan/Desktop/activelearning/data/versioned_data/raw/2
Writing (0, 0, 0)


In [7]:
print(z[5:11,10,10])

File to open:/Users/Marwan/Desktop/activelearning/data/versioned_data/raw/2
[ 0  0  0  0  0 10]


In [8]:
# No chunk file to open
print(z[300,300,300])

0
